In [ ]:
import logging
from pyspark.sql import functions as F
from pyspark.sql.types import IntegerType, StringType, DateType
from datetime import datetime

# Configure logging
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

try:
    # Load data from Unity Catalog tables
    customers_df = spark.table("genai_demo.sas.customers")
    transactions_df = spark.table("genai_demo.sas.transactions")

    # Step 1: Filter valid transactions
    valid_txns_df = transactions_df.filter((F.col("Sales") > 0) & (F.col("Product") != ""))

    # Step 2: Calculate Effective Price
    trans_step2_df = valid_txns_df.withColumn("EffectivePrice", F.col("Sales") * (1 - F.col("Discount") / 100))

    # Step 3: Calculate Total Value
    trans_step3_df = trans_step2_df.withColumn("TotalValue", F.col("EffectivePrice") * F.col("Quantity"))

    # Step 4: Join with Customers
    full_data_df = trans_step3_df.join(customers_df, "CustomerID", "left").select(
        trans_step3_df["*"], customers_df["Region"], customers_df["JoinDate"]
    )

    # Step 5: Calculate Tenure Days
    trans_step5_df = full_data_df.withColumn(
        "TenureDays", F.datediff(F.col("TransDate"), F.col("JoinDate"))
    )

    # Step 6: Assign Tenure Category
    trans_step6_df = trans_step5_df.withColumn(
        "TenureCategory",
        F.when(F.col("TenureDays") < 180, "New")
        .when(F.col("TenureDays") < 365, "Medium")
        .otherwise("Loyal")
    )

    # Step 7: Flag High Value Transactions
    trans_step7_df = trans_step6_df.withColumn("HighValueFlag", F.col("TotalValue") > 2000)

    # Step 8: Assign Product Group
    trans_step8_df = trans_step7_df.withColumn(
        "ProductGroup",
        F.when(F.col("Product").isin(["A", "C"]), "Core").otherwise("Non-Core")
    )

    # Final Data Preparation
    final_data_df = trans_step8_df

    # Step 9: Sort by ProductGroup
    sorted_final_data_df = final_data_df.orderBy("ProductGroup")

    # Step 10: Standardize TotalValue
    standardized_df = sorted_final_data_df.groupBy("ProductGroup").agg(
        F.mean("TotalValue").alias("mean_TotalValue"),
        F.stddev("TotalValue").alias("stddev_TotalValue")
    ).join(sorted_final_data_df, "ProductGroup").withColumn(
        "StandardizedTotalValue",
        (F.col("TotalValue") - F.col("mean_TotalValue")) / F.col("stddev_TotalValue")
    )

    # Step 11: Outlier Detection
    enhanced_final_data_df = standardized_df.withColumn(
        "OutlierFlag", F.when(F.abs(F.col("StandardizedTotalValue")) > 2, 1).otherwise(0)
    )

    # Write the final output to Unity Catalog
    spark.sql("DROP TABLE IF EXISTS genai_demo.sas.enhanced_final_data")
    enhanced_final_data_df.write.format("delta").mode("overwrite").saveAsTable("genai_demo.sas.enhanced_final_data")

    # Additional Analysis Outputs
    # Summary Statistics
    summary_stats_df = enhanced_final_data_df.groupBy("Region", "ProductGroup").agg(
        F.mean("TotalValue").alias("mean_TotalValue"),
        F.sum("TotalValue").alias("sum_TotalValue"),
        F.mean("Quantity").alias("mean_Quantity"),
        F.sum("Quantity").alias("sum_Quantity"),
        F.mean("Sales").alias("mean_Sales"),
        F.sum("Sales").alias("sum_Sales")
    )
    summary_stats_df.show()

    # Frequency Analysis
    freq_analysis_df = enhanced_final_data_df.groupBy("TenureCategory", "Region").count()
    freq_analysis_df.show()

    # Correlation Analysis
    correlation_df = enhanced_final_data_df.select("Sales", "Discount", "Quantity", "TotalValue").corr()
    logger.info("Correlation Analysis: %s", correlation_df)

    # Outlier Summary
    outlier_summary_df = enhanced_final_data_df.groupBy("OutlierFlag").agg(
        F.mean("Sales").alias("mean_Sales"),
        F.stddev("Sales").alias("stddev_Sales"),
        F.mean("TotalValue").alias("mean_TotalValue"),
        F.stddev("TotalValue").alias("stddev_TotalValue"),
        F.mean("Quantity").alias("mean_Quantity"),
        F.stddev("Quantity").alias("stddev_Quantity")
    )
    outlier_summary_df.show()

except Exception as e:
    logger.error("An error occurred during the ETL process: %s", str(e))
    raise
